# Terrestrial water storage changes derived form GRACE


Author:

To use this notebook, the shxarray package should be installed from https://github.com/ITC-Water-Resources/shxarray or from https://pypi.org/project/shxarray/

The data used in this example is from ITSG Level-2 datasets https://www.tugraz.at/institute/ifg/downloads/gravity-field-models/itsg-grace2018#c194128

 
Computing Terrestrial Water Storage change:

Stokes coefficients from the GRACE solutions can be converted to an equivalent water height, but several processing steps are often needed to obtain good results. The steps are described as below:

* Obtain time-anomalies by subtracting a static gravity field from the monthly solutions (more details about the static gravity filed models: https://icgem.gfz-potsdam.de/tom_longtime and https://ggos.org/item/global-gravity-field-models/)
* *To do:* Substitute the less accurate degree 2 coefficients with alternatives from a Satellite Laser Ranging solution
* *To do:* Add degree 1 variations, which are needed to resolve for the Earth's wobbly around it's center of mass
* Filter the coefficients to remove high degree noise
* Map (spherical harmonic synthesis) the results to a geographical area




In [ ]:
change to zip file format

purpose of the exrecise
3 learning outcomes of the tutorial
Water variable from earth observation - tws
2 versions
assign a certain weight to each step

%env OPENBLAS_NUM_THREADS=4

#Optionally enable autoreloading for development purposes. Note that this does not automagically reload the binary extensions
%load_ext autoreload
%autoreload 2
#also supress some warnings from pandas
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [1]:
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt
import gzip
import shxarray

Read monthly solutions Stokes coefficients form directory path

In [2]:
datadir = '~/shared/Waterflux/data/ITSG_grace2018/monthly/monthly_n96'

datadir = os.path.expanduser(datadir)

In [3]:
def load_icgem_files(datadir):

    gsm = []
    files = os.listdir(datadir)

    for file in files:
        file_path = os.path.join(datadir, file)
        gsm.append(xr.open_dataset(file_path, engine="icgem")) ## Depending on the gravitational spherical harmonic model, the engine can be = "gsmv6"
        
    dsgsm = xr.concat(gsm, dim="time")
    dsgsm = dsgsm.sortby('time')

    return dsgsm

In [4]:
ds = load_icgem_files(datadir)
ds

<xarray.Dataset>
Dimensions:  (time: 162, nm: 9409)
Coordinates:
  * nm       (nm) object MultiIndex
  * n        (nm) int64 0 1 1 1 2 2 2 2 2 3 3 ... 96 96 96 96 96 96 96 96 96 96
  * m        (nm) int64 0 0 1 -1 0 1 -1 2 -2 ... -92 93 -93 94 -94 95 -95 96 -96
  * time     (time) datetime64[ns] 2002-04-15T12:00:00 ... 2017-06-15T12:00:00
Data variables:
    cnm      (time, nm) float64 1.0 0.0 0.0 ... 2.452e-10 -2.245e-09 1.527e-09
    sigcnm   (time, nm) float64 0.0 0.0 0.0 ... 5.098e-11 2.906e-11 2.797e-11
Attributes:
    nmaxfile:   96
    nmax:       96
    format:     icgem
    norm:       fully_normalized
    gm:         398600441500000.0
    re:         6378136.3
    modelname:  ITSG-Grace2018_n96_2007-06

# Read the time-invariable static gravity field data

In [57]:
datadir = '~/shared/Waterflux/data/ITSG_grace2018/monthly/static'

datadir = os.path.expanduser(datadir)
name= "ITSG-Grace2018s.gfc"
file_path = os.path.join(datadir, name)
dsstatic = xr.open_dataset(file_path, engine="icgem" ,drop_variables=["sigcnm"])
dsstatic

<xarray.Dataset>
Dimensions:  (nm: 40401)
Coordinates:
  * nm       (nm) object MultiIndex
  * n        (nm) int64 0 1 1 1 2 2 2 2 2 ... 200 200 200 200 200 200 200 200
  * m        (nm) int64 0 0 1 -1 0 1 -1 2 ... -197 198 -198 199 -199 200 -200
Data variables:
    cnm      (nm) float64 ...
Attributes:
    nmaxfile:   200
    nmax:       200
    format:     icgem
    norm:       fully_normalized
    gm:         398600441500000.0
    re:         6378136.3
    modelname:  ITSG-Grace2018s

In [12]:
ds["dcnm"]=ds.cnm-dsstatic.cnm
ds.sel(n=2,m=0)

<xarray.Dataset>
Dimensions:  (time: 162)
Coordinates:
    nm       object (2, 0)
    n        int64 2
    m        int64 0
  * time     (time) datetime64[ns] 2002-04-15T12:00:00 ... 2017-06-15T12:00:00
Data variables:
    cnm      (time) float64 -0.0004842 -0.0004842 ... -0.0004842 -0.0004842
    sigcnm   (time) float64 1.541e-11 3.199e-11 ... 6.207e-11 7.373e-11
    dcnm     (time) float64 1.923e-10 1.174e-10 ... -1.23e-10 1.329e-10
Attributes:
    nmaxfile:   96
    nmax:       96
    format:     icgem
    norm:       fully_normalized
    gm:         398600441500000.0
    re:         6378136.3
    modelname:  ITSG-Grace2018_n96_2007-06

# Helper class which aids in extracing the gzipped data files from the Tar file without the need to extract the data to the disk
class TarExtracter:
    def __init__(self,tararchive):
        """ Opens a tararchive and keep its open acces point available to functions within this class"""
        self.tarar=tarfile.open(os.path.join(datadir,tararchive),'r:gz')
    def getmember(self,member):
        """Extract a specific (gzipped) member from an open archive, and return an open file object"""
        tarfileobj=self.tarar.extractfile(member)
        return gzip.open(tarfileobj)
    def close(self):
        self.tarar.close()
        
import tarfile

In [7]:
nmax=ds.sh.nmax
nmax

0

In [34]:
def load_icgem_files(datadir):

    gsm = []
    deg1_terms = []
    term_c20 = []
    files = os.listdir(datadir)

    for file in files:
        file_path = os.path.join(datadir, file)
        if "c20" in file:
            term_c20.append(xr.open_dataset(file_path, engine="gsmv6"))
        elif "degree1" in file:
            deg1_terms.append(xr.open_dataset(file_path, engine="gsmv6"))
                
            
        
    # dsgsm = xr.concat(gsm, dim="time")
    # dsgsm = dsgsm.sortby('time')


    if output_type == deg1:
        return xr.concat(deg1, dim="time")
    elif output_type == c20:
        return xr.concat(c20, dim="time")
    
  

In [79]:
import os
import xarray as xr

# Define your directory and file
datadir = os.path.expanduser('~/shared/Waterflux/data/ITSG_grace2018/monthly/monthly_background')
file = "model_c20_2014-05.gfc"
file_path = os.path.join(datadir, file)

ds1 = xr.open_dataset(file_path, engine="icgem" ,drop_variables=["sigcnm"])


TypeError: open_dataset() got an unexpected keyword argument 'errors'

# (To Do) Replace degree one coefficients and c20 from 
https://podaac.jpl.nasa.gov/announcements/2018-07-16_UTCSR/JPL_GRACE_Level-2_RL06_datasets_release

# Calculate terrestrial water storage in spectral domain

In [38]:
datws=ds.dcnm.sh.tws()
display(datws)

shxarray-INFO: /home/jovyan/.cache/shxarray_storage/Love/geoslurp_dump_llove.sql already exists, no need to download)


RuntimeError: Requested kernel operation is only supported for degrees {nminsup} < = n <= {nmaxsup}

# Calculate terrestrial water storage change in spatial domain globally

In [ ]:
dsgrd=datws.sh.synthesis().to_dataset(name="tws")
islice=0
dsgrd.tws[:,:,islice].plot(vmin=-0.5,vmax=0.5)
dsgrd
lats = dsgrd['lat'].values
lons = dsgrd['lon'].values

# Apply filters